# Adding Widgets

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 12)

In [2]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.plotting import ColumnDataSource
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

## Slider Widget

The example shows an action attached to a slider that updates a data source whenever the slider is moved.

In [3]:
from bokeh.layouts import column, row 
from bokeh.models import Slider

In [4]:
data = {'x':[1,2,3,4,5], 'y':[3,1,5,2,4]}
source = ColumnDataSource(data=data)

In [5]:
f1 = figure(x_range=(0, 6), y_range=(0, 6), 
            toolbar_location=None,
            width=350, height=300)
g1 = f1.circle_dot(x='x', y='y', source=source, color='indianred', size=30);

In [6]:
slider1 = Slider(start=1, end=100, step=1, value=30, title='Size')
slider1.js_link('value', g1.glyph, 'size')

In [7]:
show(column(slider1, f1))

In [8]:
show(row(f1, slider1))

## Color Picker

It is a widget to allow the user to specify an RGB color value.

In [9]:
from bokeh.models import ColorPicker

In [10]:
f2 = figure(x_range=(0, 6), y_range=(0, 6), 
            toolbar_location=None,
            width=350, height=300)
g2 = f2.square_dot(x='x', y='y', source=source, color='green', size=30)

In [11]:
picker2 = ColorPicker(title="Circles Color", color='green')
picker2.js_link('color', g2.glyph, 'fill_color')

In [12]:
show(column(picker2, f2))

## Select

It is a button that displays a drop-down list of mutually exclusive items when clicked.

In [13]:
from bokeh.models import Select

In [14]:
f3 = figure(x_range=(0, 6), y_range=(0, 6), 
            toolbar_location=None,
            width=350, height=300)
g3 = f3.scatter(x='x', y='y', source=source, color='orangered', size=30, marker='circle')

In [15]:
select3 = Select(title="Select marker", value='circle',
            options=['star','square','plus','diamond','triangle','circle'])
select3.js_link('value', g3.glyph, 'marker')            

In [16]:
show(column(select3, f3))

## Using Several Widgets

### All together

In [17]:
# Defining an empty figure
f = figure(x_range=(0, 6), y_range=(0, 6), 
            toolbar_location=None,
            width=400, height=300)

Defining the basic scatter

In [18]:
g = f.scatter(x='x', y='y', source=source,  
            color='gold', line_color='white',            
            size=30, 
            marker='circle')

Defining the widgets

In [19]:
slider = Slider(start=1, end=100, step=1, value=30, title='Size')
slider.js_link('value', g.glyph, 'size')

In [20]:
picker = ColorPicker(title="Color", color='gold')
picker.js_link('color', g.glyph, 'fill_color')

In [21]:
select = Select(title="Select marker", value='circle',
            options=['star','square','plus','diamond','triangle','circle'])
select.js_link('value', g.glyph, 'marker')   

In [22]:
show(row(f, column(select, picker, slider)))

### Quadratic Function Example

In [23]:
x4 = np.linspace(-5, 5, 1000)
y4 = x4*x4
source4 = ColumnDataSource(data=dict(x=x4, y=y4))

In [24]:
p4 = figure(y_range=(-5,5), x_range=(-5,5), width=600, height=400, title='Quadratic Function')
p4.line('x', 'y', source=source4, line_width=2, color='green', line_alpha=0.7)
show(p4)

Let's create three sliders for modifying the vertex coordinates `(h,k)` and the quadratic coefficient `a`.

In [25]:
slider_a = Slider(start=-5, end=5, value=1, step=.1, title="Quadratic Coefficient")
slider_h = Slider(start=-5, end=5, value=0, step=.1, title="Vertex x-coordinate")
slider_k = Slider(start=-5, end=5, value=0, step=.1, title="Vertex y-coordinate")

In [26]:
from bokeh.models import CustomJS

We need a simple JS code for updating the quadratic function when the user modifies the slider parameters `a`, `h`, and `k`.

In [27]:
callback = CustomJS(args=dict(source=source4, sa=slider_a, sh=slider_h, sk=slider_k), 
    code="""
    var data = source.data;
    var a = sa.value;
    var h = sh.value;
    var k = sk.value;
    var x = data['x']
    var y = data['y']
    for (var i = 0; i < x.length; i++) 
        y[i] = a * (x[i] - h)**2 + k;
    source.change.emit();
""")

In [28]:
# Linking slider values to code
slider_a.js_on_change('value', callback) 
slider_h.js_on_change('value', callback) 
slider_k.js_on_change('value', callback) 

In [29]:
show(row(p4, column(slider_a, slider_h, slider_k)))

In [30]:
show(column(slider_a, slider_h, slider_k, p4))

## Interactive legends

Legends added to Bokeh graphs can be made interactive, so clicking on the legend entries will hide or mute the corresponding glyph in a plot. You can activate this mode by setting the `click_policy` property on a legend to `hide` or `mute`.

### Automobile Dataset

We will use the Automobile Data Set [https://archive.ics.uci.edu/ml/datasets/automobile] from the UCI Machine Learning Repository [https://archive-beta.ics.uci.edu/]. It includes categorical and continuous variables. 

In [31]:
# Defining the headers
headers = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration", "num_doors", "body_style", 
           "drive_wheels", "engine_location", "wheel_base", "length", "width", "height", "curb_weight", 
           "engine_type", "num_cylinders", "engine_size", "fuel_system", "bore", "stroke", "compression_ratio", 
           "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"]

In [32]:
dfa = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data",
                  header=None, names=headers, na_values="?" )
print(dfa.shape)
dfa.head()

(205, 26)


,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,...,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,NaN,alfa-romero,gas,std,two,...,9.0,111.0,5000.0,21,27,13495.0
1,3,NaN,alfa-romero,gas,std,two,...,9.0,111.0,5000.0,21,27,16500.0
2,1,NaN,alfa-romero,gas,std,two,...,9.0,154.0,5000.0,19,26,16500.0
3,2,164.0,audi,gas,std,four,...,10.0,102.0,5500.0,24,30,13950.0
4,2,164.0,audi,gas,std,four,...,8.0,115.0,5500.0,18,22,17450.0


### Hiding glyphs

You can hide a glyph from the legend. Let's see how to do it!

Analysing the unique values of `body_style`

In [33]:
dfa.body_style.unique()

array(['convertible', 'hatchback', 'sedan', 'wagon', 'hardtop'],
      dtype=object)

In [34]:
from bokeh.palettes import Bokeh5

Creating a new plot with a title and axis labels

In [35]:
sct = figure(title="Click on legend entries to HIDE the corresponding points", 
            x_axis_label='Highway (mpg)', y_axis_label='Price',            
            height=400, width=700)

In [36]:
for option, color in zip(dfa.body_style.unique(), Bokeh5):
    sct.circle_dot(dfa.highway_mpg[dfa.body_style==option], dfa.price[dfa.body_style==option], 
                size = 15, color = color, alpha=0.7, legend_label=option)

In [37]:
sct.legend.location = "top_right"
sct.legend.click_policy="hide"
show(sct)

### Muting glyphs

You can mute a glyph from the legend. Hiding the glyph makes it vanish entirely; muting it de-emphasizes the glyph.

In [38]:
sct2 = figure(title="Click on legend entries to MUTE the corresponding points", 
            x_axis_label='Highway (mpg)', y_axis_label='Price',            
            height=400, width=700)

In [39]:
for option, color in zip(dfa.body_style.unique(), Bokeh5):
    sct2.square_pin(dfa.highway_mpg[dfa.body_style==option], dfa.price[dfa.body_style==option], 
                size = 20, color = color, alpha=0.8, legend_label=option)

In [40]:
sct2.legend.location = "top_right"
sct2.legend.click_policy="mute"
show(sct2)

## References

- https://bokeh.org/
- https://docs.bokeh.org/en/latest/docs/reference/models/glyphs/scatter.html
- https://docs.bokeh.org/en/latest/docs/user_guide/interaction.html